## Helper Functions Notebook 

This notebook includes some useful functions.

In [1]:
#IMPORTS
#libraries
import numpy as np
import time
import itertools
from treelib import Tree
import import_ipynb

#notebooks
import blockchain as bc
import payoff_matrix as pm

### Functions

The first two functions, `pickWinners()` and `pickParents()`, are exclusively used to test basic functionality of the blockchain classes (they are only used in `tests.ipynb`). They randomly pick a winner among the miners for every stage of a $T$-stage game, as well as a parent for every block. This can then be used to generate a random blockchain, allowing us to see if the code behaves as expected.

The function `drawChain()` allows us to look at the structure of the blockchain in console output. This makes visualizing what is going on a lot easier. Next, we have two quality-of-life functions called `timeElapsed()` and `printPayoffMatrix()`, which make printing passed time and payoff-matrices to the output a lot simpler. Finally, we have the function `isOnSameBranch()`. It simply checks whether two blocks $b$ and $c$ are on the same branch or not. 


In [2]:
def pickWinners(_T, _n):
    #
    #returns an array with the index of the winner of each stage 't' in a '_T'-stage game with '_n' players.
    #

    winners = np.random.randint(0, _n, _T) #'winners[t]' is the index of the winning miner of the block mined in stage 't+1'

    return winners



def pickParents(_T, _n):
    #
    #draw random parents
    #'_T' is the horizon and '_n' the number of players
    #

    parents = np.full(_T, 0)

    for t in range(1, _T):
        parents[t] = np.random.randint(0, t+1) #'parents[t]' is the index of the parent of the block mined in stage 't+1'

    return parents



def drawChain(_B):
    #
    #draw the blockchain to console output for visualisation
    #'_B' is an object of class 'Blockchain'
    #

    T = _B.horizon

    tree = Tree()
    for t in range(T+1):
        if t == 0:
            tree.create_node(f"b{_B.sequence[0].height}", f"{_B.sequence[0].height}")
        else:
            tree.create_node(f"b{_B.sequence[t].height}", f"{_B.sequence[t].height}", parent=f"{_B.sequence[t].parent}")

    print("")
    tree.show()


def timeElapsed(_start, _end):
    #
    #simply returns a string with the amount of time elapsed between '_start' and '_end'
    #
    seconds = round((_end - _start), 1)
    if seconds < 60:
        return f"{seconds} seconds"
    if seconds >= 60 and seconds < 3600:
        return f"{int(round(seconds/60, 1))} minutes"
    if seconds >= 3600:
        return f"{int(round(seconds/3600, 1))} hours"
    
    
def printPayoffMatrix(_n, _parents, _winners, _name, _T):
    #
    #prints the blockchain defined by '_parents' and '_winners' together with its payoff-matrix to output
    #
    B = bc.Blockchain(_n, _parents, _winners)
    print(f"blockchain {_name}:")
    drawChain(B)
    B.printMiner(0)
    B.printMiner(1)
    print("")
    start = time.process_time()
    print(f"payoff-matrix for blockchain {_name} in {_T}-stage game:\n\n", pm.intermediatePayoffMatrix(B, _T, _n))
    end = time.process_time()
    print("\n", timeElapsed(start, end), "\n")


    
def isOnSameBranch(_B, _b, _c):
    #
    #simply checks if two blocks are on the same branch in the blockchain'_B',
    #i.e. if we can get to block '_b' by jumping from parent to parent starting from block '_c'
    #

    isOnSameBranch = False
    t = _c

    #ITERATE BACKWARDS 
    #starting from the block mined in stage 't', which corresponds to block '_c'
    while _B.sequence[t].height != 0:
        if _B.sequence[t].parent == _b:
            isOnSameBranch = True
            break
        else:
            t = _B.sequence[t].parent

    if _b == _c: #trivial case
        isOnSameBranch = True

    return isOnSameBranch


The next two functions are exclusively used in the notebook `conjectures.ipynb`. They generate all possible sequences of parents and winners. This is used to iterate through all possible blockchains to test the conjectures.

In [3]:
def generatePossibleParents(_T):
    #
    #generates all possible sequences of parents for a game with '_T' stages
    #returns an array
    #

    possible_parents = [] #list of lists of parents

    #GENERATE EVERY SEQUENCE
    blocks = np.arange(_T)
    for element in itertools.product(blocks, repeat=_T):

        #CHECK IF ELEMENT IS LEGAL
        legal = True
        for i in range(len(element)):
            if element[i] > i: #parent must have strictly lower index
                legal = False
                break
        if legal == False:
            continue

        possible_parents.append(list(element)) #append legal combination

    possible_parents = np.array(possible_parents)

    return possible_parents


def generatePossibleWinners(_T, _n):
    #
    #generates all possible sequences of winners for a game with '_n' miners and '_T' stages
    #returns an array
    #

    possible_winners = [] #list of lists of winners

    #GENERATE EVERY SEQUENCE
    miners = np.arange(_n)
    for combination in itertools.combinations_with_replacement(miners, _T):
        for permutation in itertools.permutations(combination, _T):
            if permutation[0] == 1: #w.l.o.g. m0 wins the first stage, hence a situation where m1 wins is skipped
                #we just need to make sure to check the strategies for BOTH miners later, since we exclude half the game-tree here
                continue
            possible_winners.append(list(permutation))

    possible_winners = np.array(possible_winners)
    possible_winners = np.unique(possible_winners, axis=0) #kick duplicate entries from array, this is needed because the permutations consider winning stage 2 and stage 1 different from winning stage 1 and stage 2, even tough they are in fact the same

    return possible_winners
